In [ ]:
# !pip install -U nolds matplotlib numpy pandas mpld3 statsmodels scikit-learn
%pip install pmdarima --user 

In [5]:
# %matplotlib inline
import mpld3
mpld3.enable_notebook()
%matplotlib auto
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima_process import arma_generate_sample, arma_acf
from scipy.stats import norm
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
os.getcwd()

Using matplotlib backend: Qt5Agg


'c:\\Users\\tonyt\\timeseries-codebase\\LinearTSAnalysis'

## Utils

In [3]:
def plot_timeseries(xV, get_histogram=False, title='', savepath=''):
    # #plot timeseries
    fig, ax = plt.subplots(1, 1, figsize=(14, 8))
    ax.plot(xV, marker='x', linestyle='--', linewidth=2);
    ax.set_xlabel('time')
    ax.set_ylabel('value')
    if len(title) > 0:
        ax.set_title(title, x=0.5, y=1.0);
    plt.tight_layout()
    if len(savepath) > 0:
        plt.savefig(f'{savepath}/{title}_xM.jpeg')
    # #plot histogram
    if get_histogram:
        fig, ax = plt.subplots(1, 1, figsize=(14, 8))
        ax.hist(xV, alpha=0.8, rwidth=0.9);
        ax.set_xlabel('value')
        ax.set_title('Histogram')
        plt.tight_layout()
        if len(title) > 0:
            ax.set_title(title, x=0.5, y=1.0);
        plt.tight_layout()
        if len(savepath) > 0:
            plt.savefig(f'{savepath}/{title}_hist.jpeg')



def read_datfile(path):
    xV = np.loadtxt(path)
    return xV

def rolling_window(xV, window):
    '''
    returns moving average of a time series xV 
    with length of window
    '''
    xV = xV.flatten()
    return np.convolve(xV, np.ones(window)/window, mode='same') 

def polynomial_fit(xV, p):
    '''
    fit to a given time series with a polynomial of a given order.
    :param xV: vector of length 'n' of the time series
    :param p: the order of the polynomial to be fitted
    :return: vector of length 'n' of the fitted time series
    '''
    n = xV.shape[0]
    xV = xV[:]
    if p > 1:
        tV = np.arange(n)
        bV = np.polyfit(x=tV, y=xV, deg=p)
        muV = np.polyval(p=bV, x=tV)
    else:
        muV = np.full(shape=n, fill_value=np.nan)
    return muV


def seasonal_components(xV, period):
    '''
    computes the periodic time series comprised of repetetive
    patterns of seasonal components given a time series and the season
    (period).
    '''
    n = xV.shape[0]
    sV = np.full(shape=(n,), fill_value=np.nan)
    monV = np.full(shape=(period,), fill_value=np.nan)
    for i in np.arange(period):
        monV[i] = np.mean(xV[i:n:period])
    monV = monV - np.mean(monV)
    for i in np.arange(period):
         sV[i:n:period] = monV[i] * np.ones(shape=len(np.arange(i, n, period)))
    return sV



def generate_arma_ts(phiV, thetaV, n, sdnoise=1):
    '''
    Generate an ARMA(p,q) time series of length 'n' with Gaussian input noise.
    Note that phiV = [phi(0) phi(1) ... phi(p)]' and phi(0) is the constant
    term, and thetaV = [theta(1) ... theta(q)]'.
    sdnoise is the SD of the input noise (if left out then sdnoise=1).
    The generating ARMA(p,q) process reads
    x(t) = phi(0) + phi(1)*x(t-1) + ... + phi(p)*x(t-p) +
            +z(t) - theta(1)*z(t-1) + ... - theta(q)*z(t-p),
    z(t) ~ WN(0,sdnoise^2)
    '''
    phiV = np.array(phiV)
    thetaV = np.array(thetaV)
    ar_params = np.r_[1, -phiV[:]]  # add zero lag
    ma_params = np.r_[1, thetaV[:]]  # add zero lag
    xV = arma_generate_sample(ar=ar_params, ma=ma_params, nsample=n, scale=sdnoise, burnin=100)
    return xV
    # q = len(thetaV)
    # p = len(phiV) - 1
    # pq = np.max(p, q)
    # ntrans = 100 + pq
    # phiV = phiV[:]
    # thetaV = thetaV[:]
    # if p > 0:
    #     root_arV = np.roots(np.r_[1, -phiV[1:]])
    #     if np.any(np.abs(root_arV) >= 1):
    #         print(f'The AR({p}) part of the process is not stationary.\n')
    # if q > 0:
    #     root_maV = np.roots(np.r_[1, -thetaV[1:]])
    #     if np.any(np.abs(root_maV) >= 1):
    #         print(f'The MA({p}) part of the process is not stationary.\n')
    # x0V = sdnoise * np.random.normal(size=(pq, 1))
    # zV = sdnoise * np.random.normal(size=(n+ntrans, 1))
    # xV = np.full(shape=(n+ntrans, 1), fill_value=np.nan)
    # xV[:pq] = x0V
    # if p == 0:
    #     for i in np.arange(pq+1, n+ntrans):
    #         xV[i] = phiV[0] + zV[i] - thetaV * np.flipud(zV[i - q:i])
    # elif q == 0:
    #     for i in np.arange(pq+1, n+ntrans):
    #         xV[i] = phiV[0] + phiV[1: p+1] * np.flipud(xV[i-p:i-1]) + zV[i]
    # else:
    #     for i in np.arange(pq+1, n+ntrans):
    #         xV[i] = phiV[0] + phiV[1:p+1] * np.flipud(xV[i-p:i-1]) + zV[i] - thetaV * np.flipud(zV[i-q:i-1])
    # xV = xV[ntrans + 1:]
    # return xV

def add_stochastic_trend(xV):
    '''
    adds a stochastic trend to a given time series (for
    simulating purposes). The time series of stochastic trend is generated by
    simulating a smoothed random walk time series of the same length as that of the
    given time series.
    :param xV: vector of length 'n' of the given time series
    :return: vector of length 'n' of the sum of the given time series and a stochastic trend
    '''
    xV = xV[:]
    n = xV.shape[0]
    maorder = np.round(n // 5)
    x_std = np.std(xV)
    zV = 0.1 * x_std * np.random.normal(0, 1, n)
    zV = np.cumsum(zV)
    wV = rolling_window(zV, window=maorder)
    yV = xV + wV
    return yV

def add_seasonality(xV, period):
    '''
    adds a seasonal component to a given time series (for
    simulating purposes). The time series of seasonality is generated by
    cosine function of a given period 'per' and amplitude equal to the
    standard deviation of the given time series.
    '''
    n = xV.shape[0]
    xV = xV[:]
    x_sd = np.std(xV)
    zV = x_sd * np.cos(2*np.pi*np.arange(n)/period)
    return xV + zV


def armacoefs2autocorr(phiV, thetaV, lags=10):
    '''
    Theoretical autocorrelation function of an ARMA process.
    phiV: The coefficients for autoregressive lag polynomial, not including zero lag.
    thetaV : array_like, 1d
        The coefficients for moving-average lag polynomial, not including zero lag.
    '''
    phiV, thetaV = np.array(phiV), np.array(thetaV)
    phiV = np.r_[1, -phiV] # add zero lag
    thetaV = np.r_[1, thetaV] # #add zero lag
    acf_ = arma_acf(phiV, thetaV, lags=lags)
    fig, ax = plt.subplots(1, 1, figsize=(14, 8))
    ax.scatter(np.arange(1, lags), acf_[1:], marker='o')
    ax.set_xlabel('Lags')
    ax.set_xticks(np.arange(1, lags))
    ax.set_yticks(np.arange(-1, 1, 0.1))
    ax.set_title('ACF', fontsize=14)
    ax.grid(linestyle='--', linewidth=0.5, alpha=0.15)
    plt.show()
#     for t in np.arange(lags):
#         ax.axvline(t, ymax=acf_[t], color='red', alpha=0.3);

# def macoef2autocorr(phiV, thetaV, lags=10):
#     from statsmodels.tsa.arima_process import arma_pacf
#     pacf_ = arma_pacf(phiV, thetaV, lags=10)
#     fig, ax = plt.subplots(1, 1)
#     ax.scatter(np.arange(lags), pacf_, marker='o');
#     for t in np.arange(lags):
#         ax.axvline(t, ymax=pacf_[t], color='red', alpha=0.3);
        

def plot_3d_attractor(xM):
    '''
    plot 3d attractor
    '''
    fig = plt.figure(figsize=(14, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xM[:, [0]], xM[:, [1]], xM[:, [2]])
    plt.show()

def embed_data(xV, m=3, tau=1):
    """Time-delay embedding.
    Parameters
    ----------
    x : 1d-array, shape (n_times)
        Time series
    m : int
        Embedding dimension (order)
    tau : int
        Delay.
    Returns
    -------
    embedded : ndarray, shape (n_times - (order - 1) * delay, order)
        Embedded time-series.
    """
    N = len(xV)
    nvec = N - (m-1)*tau
    xM = np.zeros(shape=(nvec, m))
    for i in np.arange(m):
        xM[:, m-i-1] = xV[i*tau:nvec+i*tau]
    return xM

def get_acf(xV, lags=10, alpha=0.05, show=True):
    '''
    calculate acf of timeseries xV to lag (lags) and show
    figure with confidence interval with (alpha)
    '''
    acfV = acf(xV, nlags=lags)[1:]
    z_inv = norm.ppf(1-alpha/2)
    upperbound95 = z_inv / np.sqrt(xV.shape[0])
    lowerbound95 = -upperbound95
    if show:
        fig, ax = plt.subplots(1, 1, figsize=(14,8))
        ax.plot(np.arange(1, lags+1), acfV, marker='o')
        ax.axhline(upperbound95, linestyle='--', color='red', label=f'Conf. Int {(1-alpha)*100}%')
        ax.axhline(lowerbound95, linestyle='--', color='red')
        ax.set_title('ACF')
        ax.set_xlabel('Lag')
        ax.set_xticks(np.arange(1, lags+1))
        ax.grid(linestyle='--', linewidth=0.5, alpha=0.15)
        ax.legend()
    return acfV  
	
def get_pacf(xV, lags=10, alpha=0.05, show=True):
    '''
    calculate pacf of timeseries xV to lag (lags) and show
    figure with confidence interval with (alpha)
    '''
    pacfV = pacf(xV, nlags=lags)[1:]
    z_inv = norm.ppf(1-alpha/2)
    upperbound95 = z_inv / np.sqrt(xV.shape[0])
    lowerbound95 = -upperbound95
    if show:
        fig, ax = plt.subplots(1, 1, figsize=(14,8))
        ax.plot(np.arange(1, lags+1), pacfV, marker='o')
        ax.axhline(upperbound95, linestyle='--', color='red', label=f'Conf. Int {(1-alpha)*100}%')
        ax.axhline(lowerbound95, linestyle='--', color='red')
        ax.set_title('PACF')
        ax.set_xlabel('Lag')
        ax.set_xticks(np.arange(1, lags+1))
        ax.grid(linestyle='--', linewidth=0.5, alpha=0.15)
        ax.legend()
    return pacfV  

def portmanteau_test(xV, maxtau, show=False):
    '''
    PORTMANTEAULB hypothesis test (H0) for independence of time series:
    tests jointly that several autocorrelations are zero.
    It computes the Ljung-Box statistic of the modified sum of 
    autocorrelations up to a maximum lag, for maximum lags 
    1,2,...,maxtau. 
    '''
    ljung_val, ljung_pval = acorr_ljungbox(xV, lags=maxtau)
    if show:
        fig, ax = plt.subplots(1, 1)
        ax.scatter(np.arange(len(ljung_pval)), ljung_pval)
        ax.axhline(0.05, linestyle='--', color='r')
        ax.set_title('Ljung-Box Portmanteau test')
        ax.set_yticks(np.arange(0, 1.1))
        plt.show()
    return ljung_val, ljung_pval

def fit_arima_model(xV, p, q, d=0, show=False):
    '''
    fit ARIMA(p, d, q) in xV
    returns: summary (table), fittedvalues, residuals, model, AIC
    '''
    model = ARIMA(xV, order=(p, d, q)).fit()
    summary = model.summary()
    fittedvalues = model.fittedvalues
    fittedvalues = np.array(fittedvalues).reshape(-1, 1)
    resid = model.resid
    if show:
        fig, ax = plt.subplots(1, 1, figsize=(14, 8))
        ax.plot(xV, label='Original', color='blue')
        ax.plot(fittedvalues, label='FittedValues', color='red', linestyle='--', alpha=0.9)
        ax.legend()
        ax.set_title(f'ARIMA({p}, {d}, {q})')
        fig, ax = plt.subplots(2, 1, figsize=(14, 8))
        ax[0].hist(resid, label='Residual')
        ax[1].scatter(np.arange(len(resid)),resid)
        plt.title('Residuals')
        plt.legend()
    return summary, fittedvalues, resid, model, model.aic

def calculate_fitting_error(xV, model, Tmax=20, show=False):
    '''
    calculate fitting error with NRMSE for given model in timeseries xV
    till prediction horizon Tmax
    returns:
    nrmseV
    preds: for timesteps T=1, 2, 3
    '''
    nrmseV = np.full(shape=Tmax, fill_value=np.nan)
    nobs = len(xV)
    xV_std = np.std(xV)
    vartar = np.sum((xV - np.mean(xV)) ** 2)
    predM = []
    tmin = np.max([len(model.arparams), len(model.maparams), 1]) # start prediction after getting all lags needed from model
    for T in np.arange(1, Tmax):
        errors = []
        predV = np.full(shape=nobs, fill_value=np.nan)
        for t in np.arange(tmin, nobs-T):
            pred_ = model.predict(start=t, end=t+T-1, dynamic=True)
            # predV.append(pred_[-1])
            ytrue = xV[t+T-1]
            predV[t+T-1] = pred_[-1]
            error = pred_[-1] - ytrue
            errors.append(error)
        predM.append(predV)
        errors = np.array(errors)
        mse = np.mean(np.power(errors, 2))
        rmse = np.sqrt(mse)
        nrmseV[T] = (rmse / xV_std)
        # nrmseV[T] = (np.sum(errors**2) / vartar)
    if show:
        fig, ax = plt.subplots(1, 1, figsize=(14, 8))
        ax.plot(np.arange(1, Tmax), nrmseV[1:], marker='x', label='NRMSE');
        ax.axhline(1, color='red', linestyle='--');
        ax.set_title('Fitting Error')
        ax.legend()
        ax.set_xlabel('T')
        ax.set_xticks(np.arange(1, Tmax))
        plt.show()
        # #plot multistep prediction for T=1, 2, 3
        fig, ax = plt.subplots(1, 1, figsize=(14,8))
        ax.plot(xV, label='original')
        colors = ['red', 'green', 'black']
        for i, preds in enumerate(predM[:3]):
            ax.plot(preds, color=colors[i], linestyle='--', label=f'T={i+1}', alpha=0.7)
        ax.legend(loc='best')
        plt.show()
    return nrmseV, predM


def predict_multistep(model, Tmax=10, show=False):
    tmin = np.max([len(model.arparams), len(model.maparams), 1]) # start prediction after getting all lags needed from model
    preds = model.predict(start=tmin, end=Tmax, dynamic=True)
    if show:
        fig, ax = plt.subplots(1, 1, figsize=(14, 8))
        ax.plot(preds)
        ax.set_title('Multistep prediction')
        ax.set_xlabel('T')
        plt.show
    return preds

def gaussianisation(data):
    '''
    transform a variable of any distribution 
    into normal
    '''
    sort_ind = np.argsort(data)
    gaussian_data = np.random.normal(0, 1, size=data.shape[0])
    gaussian_data_ind = np.argsort(gaussian_data)
    g_d_sorted = gaussian_data[gaussian_data_ind]
    y = np.zeros(shape=data.shape[0])
    for i in np.arange(data.shape[0]):
        y[i] = g_d_sorted[sort_ind[i]]
    return y

def get_nrmse(target, predicted):
    se = (target - predicted)**2
    mse = np.mean(se)
    rmse = np.sqrt(mse)
    return rmse/np.std(target)

## GENERATE DATA

In [7]:
n = 1000
sd_noise = 1
mux = 0
###white noise
# xV = np.random.normal(0, sd_noise, n) + mux
###random walk
# xV = np.cumsum(xV)
###AR(1)
xV = generate_arma_ts(phiV=[0.9], thetaV=[0.], n=n)
plot_timeseries(xV)


In [8]:
_ = get_acf(xV)

In [9]:
plot_acf(xV, zero=False, lags=10);
plot_pacf(xV, zero=False, lags=10);


C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\graphics\tsaplots.py:348: FutureWarning: The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.
  warnings.warn(


In [6]:
# #add stochastic trend
xV = add_stochastic_trend(xV)
plot_timeseries(xV)
plot_acf(xV, zero=False, lags=10);


NameError: name 'xV' is not defined

In [8]:
# #add seasonality
perseason = 12
xV = add_seasonality(xV, period=perseason)
plot_timeseries(xV)
plot_acf(xV, zero=False, lags=10);

## REMOVE TREND

In [9]:
window = 15
ma = rolling_window(xV=xV, window=window)
plt.figure()
plt.plot(ma, linestyle='--')
plt.plot(xV, alpha=0.5)
plt.plot(xV-ma, alpha=0.5)
plt.legend([f'MA', 'Original', 'Detrended'])

In [11]:
xV_df = pd.DataFrame(xV)
xV_df.rolling(window=window, min_periods=1).mean().plot()


<AxesSubplot:>

In [12]:
###polynomial fit
p = 3;
pol = polynomial_fit(xV, p=p)
plt.plot(pol)
plt.plot(xV, alpha=0.5)
plt.plot(xV-pol, alpha=0.5)
plt.legend([f'Pol({p})', 'Original', 'Detrended'])

In [13]:
detrended = xV - ma;
plt.plot(detrended)

In [14]:
plot_acf(detrended, zero=False);

In [15]:
###first diffs
fd = np.diff(xV)

In [17]:
plt.plot(fd)

## REAL DATA

In [12]:
os.chdir('C:\\Users\\tonyt\\timeseries-codebase')
df = pd.read_csv('./data/BTCUSDT.csv')

In [13]:
df.head()

,time,close
0,2017-08-18,4285.08
1,2017-08-19,4108.37
2,2017-08-20,4139.98
3,2017-08-21,4086.29
4,2017-08-22,4016.00


In [14]:
df.set_index(pd.to_datetime(df['time']), inplace=True)
df.drop('time', axis=1, inplace=True)

In [15]:
df.plot()

<AxesSubplot:xlabel='time'>

In [16]:
df = df['01-01-2020':'12-31-2020']
df.plot();

In [10]:
xV = df.values

NameError: name 'df' is not defined

In [28]:
get_acf(xV, );

In [29]:
xV = np.log(df).diff().bfill()

In [30]:
xV.plot();

In [31]:
xV = xV.values

In [32]:
plot_acf(xV, zero=False);

In [33]:
plot_pacf(xV, zero=False);

C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\graphics\tsaplots.py:348: FutureWarning: The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.
  warnings.warn(


In [45]:
ljung_val, ljung_pval = portmanteau_test(xV, maxtau=10, show=True)

ValueError: x and y must be the same size

In [49]:
summary, fittedvalues, resid, model, aic = fit_arima_model(xV=xV, p=1, q=1, d=0, show=True)
aic

No handles with labels found to put in legend.


-1285.108801674428

In [50]:
best_aic = np.inf
best_p = None
best_q = None
for p in np.arange(1, 6, dtype=np.int):
    for q in np.arange(0, 6, dtype=np.int):
        try:
            _, _, _, _, aic = fit_arima_model(xV=xV, p=p, q=q, d=0, show=False)
        except ValueError as err:
            print(f'p:{p} - q:{q} - err:{err}')
            continue
        print(f'p:{p} - q:{q} - aic:{aic}')
        if aic < best_aic:
            best_p = p
            best_q = q
            best_aic = aic
print(f'AR order:{best_p}')
print(f'MA order:{best_q}')
print(f'Best AIC:{best_aic}')

<ipython-input-50-2f67e1cd38fd>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  for p in np.arange(1, 6, dtype=np.int):
<ipython-input-50-2f67e1cd38fd>:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; 

p:1 - q:0 - aic:-1282.3256280574046
p:1 - q:1 - aic:-1285.108801674428
p:1 - q:2 - aic:-1283.5449820510814
p:1 - q:3 - aic:-1281.8094757983238


C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


p:1 - q:4 - aic:-1282.8280114612326
p:1 - q:5 - aic:-1280.862652532775
p:2 - q:0 - aic:-1282.5977411890121


<ipython-input-50-2f67e1cd38fd>:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  for q in np.arange(0, 6, dtype=np.int):


...

C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\tonyt\anaconda3\lib\site-packa

p:5 - q:3 - aic:-1277.6884673203124


C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


p:5 - q:4 - aic:-1275.5412500012287
p:5 - q:5 - aic:-1255.9860697727697
AR order:1
MA order:1
Best AIC:-1285.108801674428


C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [51]:
summary, fittedvalues, resid, model, aic = fit_arima_model(xV=xV, p=best_p, q=best_q, d=0, show=True)
summary

No handles with labels found to put in legend.


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  366
Model:                 ARIMA(1, 0, 1)   Log Likelihood                 646.554
Date:                Wed, 12 Jan 2022   AIC                          -1285.109
Time:                        17:17:47   BIC                          -1269.498
Sample:                             0   HQIC                         -1278.906
                                - 366                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0038      0.003      1.488      0.137      -0.001       0.009
ar.L1         -0.6571      0.180     -3.645      0.000      -1.010      -0.304
ma.L1          0.4962      0.195      2.546      0.011       0.114       0.878
sigma2         0.0017   3.27e-05     52.228      0.000       0.002       0.002
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):             55183.71
Prob(Q):                              0.91   Prob(JB):                         0.00
Heteroskedasticity (H):               0.30   Skew:                            -4.82
Prob(H) (two-sided):                  0.00   Kurtosis:                        62.38
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [55]:
nrmseV, predM = calculate_fitting_error(xV, model, Tmax=10, show=True)

KeyError: 1

In [53]:
xV = np.log(df).diff().bfill()
xV_sq = xV ** 2
xV_sq.plot();

In [54]:
plot_acf(xV_sq, zero=False);
# utils.get_pacf(xV_sq)

In [56]:
plot_pacf(xV_sq, zero=False);

C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\graphics\tsaplots.py:348: FutureWarning: The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.
  warnings.warn(


## GAUSSIANIZE DATA

In [57]:
temp = np.random.uniform(-1, 1, 5000)
temp_gaussian = gaussianisation(temp)
fig, ax = plt.subplots(1, 2)
ax[0].hist(temp);
ax[1].hist(temp_gaussian);

In [58]:
xV_gaussian = gaussianisation(xV.values.reshape(-1,))
plt.hist(xV_gaussian)

(array([ 4., 11., 32., 70., 83., 89., 36., 26., 11.,  4.]),
 array([-2.77387376, -2.19453273, -1.6151917 , -1.03585067, -0.45650965,
         0.12283138,  0.70217241,  1.28151344,  1.86085446,  2.44019549,
         3.01953652]),
 <BarContainer object of 10 artists>)

In [59]:
plot_acf(xV_gaussian, zero=False);

In [60]:
xVsq_gaussian = gaussianisation(xV_sq.values.reshape(-1,))
plt.hist(xVsq_gaussian);

In [61]:
plot_acf(xVsq_gaussian, zero=False);

In [62]:
plot_pacf(xVsq_gaussian, zero=False);

C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\graphics\tsaplots.py:348: FutureWarning: The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.
  warnings.warn(


## RESHUFFLE

In [63]:
xV_gaussian_rp = xV_gaussian[np.random.permutation(np.arange(xV_gaussian.shape[0]))]
plot_acf(xV_gaussian_rp, zero=False);

## PORTMANTEAU TEST

In [66]:
ljung_val, ljung_pval = portmanteau_test(xV, maxtau=10, show=True)

ValueError: x and y must be the same size

## FIT MODEL

In [67]:
summary, fittedvalues, resid, model, aic = fit_arima_model(xV=xV, p=1, q=0, d=0, show=True)

C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
No handles with labels found to put in legend.


In [68]:
nrmseV, predM = calculate_fitting_error(xV, model, Tmax=10, show=True)

KeyError: 1

In [69]:
prop = 0.7
split_point = int(prop*xV.shape[0]) 
train_xV, test_xV = xV[:split_point], xV[split_point:]
summary, fittedvalues, resid, model, aic = fit_arima_model(xV=train_xV, p=1, q=0, d=0, show=True)

C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
C:\Users\tonyt\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
No handles with labels found to put in legend.


In [80]:
Tmax = 50
forecast, sterror, confint = model.forecast(Tmax)
plt.plot(forecast, label='Forecast')
plt.fill_between(np.arange(Tmax),confint[:,0], confint[:,1], alpha=0.3, color='c', label='Conf.Int')
plt.plot(test_xV[:Tmax], linestyle='--', color='b', label='Original')
plt.legend()

ValueError: too many values to unpack (expected 3)

## Out of sample predictions using pm.auto_arima

In [81]:
prop = 0.7
split_point = int(prop*xV.shape[0]) 
train_xV, test_xV = xV[:split_point], xV[split_point:]
model = pm.auto_arima(train_xV, 
                          start_p=0, start_q=0,d=0, max_p=5, 
                          max_q=5, start_P=0, D=None, start_Q=0, max_P=5, 
                          max_D=1, max_Q=5,stepwise=True,seasonal=False)
print(model.summary())
plt.hist(model.resid());

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  256
Model:               SARIMAX(1, 0, 1)   Log Likelihood                 429.805
Date:                Wed, 12 Jan 2022   AIC                           -853.609
Time:                        18:29:09   BIC                           -842.974
Sample:                             0   HQIC                          -849.332
                                - 256                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.7486      0.141     -5.316      0.000      -1.025      -0.473
ma.L1          0.5756      0.164      3.517      0.000       0.255       0.896
sigma2         0.0020   3.63e-05     56.179      0.0

## multistep oos prediction

In [90]:
def predict_oos_multistep(model:ARIMA, Tmax=10, return_conf_int=True, alpha=0.05, show=True):
    '''
    out of sample predictions starting from last train values
    '''
    if return_conf_int:
        preds, conf_bounds = model.predict(n_periods=Tmax, return_conf_int=return_conf_int, alpha=alpha)
    else:
        preds = model.predict(n_periods=Tmax, return_conf_int=return_conf_int, alpha=alpha)
        conf_bounds = []
    if show:
        fig, ax = plt.subplots(1, 1)
        ax.plot(np.arange(1, Tmax+1), preds)
    return preds, conf_bounds
model.

SyntaxError: invalid syntax (<ipython-input-90-71550715d007>, line 14)

In [83]:
in_preds = model.predict_in_sample(dynamic=False)

In [84]:
Tmax = 30
return_conf_int = True
alpha = 0.05
preds, conf_bounds = predict_oos_multistep(model, Tmax=Tmax, return_conf_int=return_conf_int, alpha=alpha, show=False)
plt.figure();
plt.plot(np.arange(1, Tmax+1),preds, label='predictions');
plt.plot(np.arange(1, Tmax+1),test_xV[:Tmax], label='original');
if return_conf_int:
    plt.fill_between(np.arange(1, Tmax+1), conf_bounds[:, 0], conf_bounds[:, 1], color='green', alpha=0.3)
plt.legend();

NameError: name 'predict_oos_multistep' is not defined

## rolling oos prediction

In [ ]:
preds = []
bounds = []
return_conf_int = True
alpha = 0.05
for i in test_xV:
    prediction, conf_bounds = model.predict(n_periods=1, return_conf_int=return_conf_int, alpha=alpha)
    model.update(i)
    preds.append(prediction[0])
    bounds.append(conf_bounds[0])
plt.figure();
plt.plot(preds, label='predictions', linestyle='--', alpha=0.3);
plt.plot(test_xV, label='original', alpha=0.7);
if return_conf_int:
    bounds = np.array(bounds)
    plt.fill_between(np.arange(len(test_xV)), bounds[:, 0], bounds[:, 1], alpha=0.3, color='green')
plt.legend();

## BTCUSDT price prediction


In [ ]:
os.chdir('d:/timeserieslab/')
df = pd.read_csv('./data/BTCUSDT.csv')
df.set_index(pd.to_datetime(df['time']), inplace=True)
df.drop('time', axis=1, inplace=True)
df = df['01-01-2020':'12-31-2021']
df.plot();

In [ ]:
logreturns = df.apply(lambda x: np.log(x)).diff().bfill()
logreturns.plot();
plot_acf(logreturns.values, zero=False)
plot_pacf(logreturns.values, zero=False)

In [ ]:
train_split = '12-01-2020'

train_xV, test_xV = logreturns[:train_split], logreturns[train_split:]
model = pm.auto_arima(train_xV, 
                          start_p=0, start_q=0,d=0, max_p=5, 
                          max_q=5, start_P=0, D=None, start_Q=0, max_P=5, 
                          max_D=1, max_Q=5,stepwise=True,seasonal=False)
print(model.summary())

plt.figure()
plt.hist(model.resid());
plt.figure()
plt.plot(train_xV, label='train data');
insample_preds = model.predict_in_sample()
insample_preds_df = pd.DataFrame(index=train_xV.index, data=insample_preds)
plt.plot(insample_preds_df, label='fitted values');
plt.legend();


In [ ]:
preds = []
for i in test_xV.values:
    prediction = model.predict(n_periods=1)[0]
    model.update(i)
    preds.append(prediction)
preds = np.array(preds)
preds_df = pd.DataFrame(index=test_xV.index, data=preds)

plt.figure();
plt.plot(preds_df, label='predictions', linestyle='--', alpha=0.3);
plt.plot(test_xV, label='original', alpha=0.7);
plt.legend();

In [ ]:
results = pd.concat([preds_df, test_xV], axis=1)
results.columns = ['pred', 'true']
results['hit'] = (results['pred'] > 0) == (results['true'] > 0)
results.head()

In [ ]:
print(results['hit'].mean())

In [ ]:
results.resample('M').agg({'hit':'mean'})

In [ ]:
get_nrmse(target=test_xV.values, predicted=preds)